### from https://youtu.be/L4p-ju44spQ

# 케라스 Word2Vec 구현

* 참고: https://wikidocs.net/69141

### 전처리

In [ ]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                            remove=('headers', 'footers', 'quites'))

In [ ]:
documents = dataset.data
documents[1]

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
import pandas as pd

news_df = pd.DataFrame({'article':documents})
len(news_df)

In [ ]:
# news_df
news_df[news_df['article'].apply(len) < 0]

In [ ]:
import re
def clean_text(doc):
  pattern = '[^a-zA-Z\s]'
  text = re.sub(pattern, '', doc)
  return text

news_df['clean_text'] = news_df['article'].apply(clean_text)
news_df

In [ ]:
from nltk.corpus import stopwords

def clean_stopword(doc):
  stop_words = stopwords.words('english')
  return ' '.join([w.lower() for w in doc.split() if w not in stop_words and len(w) > 3])

news_df['clean_stopword'] = news_df['clean_text'].apply(clean_stopword)
news_df

In [ ]:
from nltk.tokenize import word_tokenize
def tokenize(doc):
  return word_tokenize(doc)

tokenzied_news = news_df['clean_stopword'].apply(tokenize)
tokenzied_news = tokenzied_news.to_list()

import numpy as np

drop_news = [index for index, sentence in enumerate(tokenzied_news) if len(sentence) <= 1]
news_texts = np.delete(tokenzied_news, drop_news, axis=0)
len(news_texts)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

news_20000 = news_texts[:20000]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(news_20000)

In [ ]:
idx2word = {value:key for key, value in tokenizer.word_index.items()}
sequences = tokenizer.texts_to_sequences(news_20000)
len(tokenizer.word_index)

In [ ]:
print(sequences[0])

### Skipgram

#### Skipgram 전처리

* 네거티브 샘플링(Negative Sampling)

  + Word2Vec은 출력층이 내놓는 값에 소프트맥스 함수를 적용해 확률값으로 변환한 후 이를 정답과 비교해 역전파(backpropagation)
  + 소프트맥스를 적용하려면 분모에 해당하는 값, 즉 중심단어와 나머지 모든 단어의 내적을 한 뒤, 이를 다시 exp 계산을 하는데 전체 단어가 많을 경우 엄청난 계산량 발생
  + 네거티브 샘플링은 소프트맥스 확률을 구할 때 전체 단어를 대상으로 구하지 않고, 일부 단어만 뽑아서 계산을 하는 방식
  + 네거티브 샘플링 동작은 사용자가 지정한 윈도우 사이즈 내에 등장하지 않는 단어(negative sample)를 5~20개 정도 뽑고, 이를 정답단어와 합쳐 전체 단어처럼 소프트맥스 확률을 계산하여 파라미터 업데이트

In [ ]:
from tensorflow.keras.preprocessing.sequence import skipgrams
vocab_size = len(tokenizer.word_index) + 1
skip_grams_sample = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in sequences[:10]]

In [ ]:
paris, labels = skip_grams_sample[0][0], skip_grams_sample[0][1]
print(paris)
print(labels)

In [ ]:
for i in range(5):
  print('{}({}), {}({}) -> {}'.format(
      idx2word[paris[i][0]], paris[i][0],
      idx2word[paris[i][1]], paris[i][1],
      labels[i]
  ))

#### Skipgram 모델 구성

In [ ]:
skip_grams = [skipgrams(seq, vocabulary_size=vocab_size, window_size=10) for seq in sequences[:10]]

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dot
from tensorflow.keras.utils import  plot_model

In [ ]:
embed_size = 50
def word2vec():
  target_inputs = Input(shape=(1,), dtype='int32')
  target_embedding = Embedding(vocab_size, embed_size)(target_inputs)

  context_inputs = Input(shape=(1,), dtype='int32')
  context_embedding = Embedding(vocab_size, embed_size)(context_inputs)

  dot_product = Dot(axes=2)([target_embedding, context_embedding])
  dot_product = Reshape((1,), input_shape=(1,1))(dot_product)

  output = Activation('sigmoid')(dot_product)

  model = Model(inputs=[target_inputs, context_inputs], outputs=output)
  model.compile(loss='binary_crossentropy', optimizer='adam')

  return model

In [ ]:
model = word2vec()
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
model.fit 

### CBOW


#### CBOW 전처리

#### CBOW 모델 구성